# Scan precursor

In [1]:
%%capture

# Important header information
naluhelperdir = '../../../../utilities'
import sys
sys.path.insert(1, naluhelperdir)

# Change some display properties
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
#import windspectra
import os.path
from matplotlib.lines import Line2D
%matplotlib inline

In [2]:
# Target TI conditions
target = {}
target['ws']    = 9.0
target['ti']    = 0.1804 #0.111     # From Mithu's slides (placeholder for now)
target['veer']  = 5.542     # Degrees from 10m - 169m
target['alpha'] = 0.0898
target['utau']  = 0.486
target['hh']    = 91.0      # Hub height
target['hti']   = 60.0      # Hub height

In [3]:
HHplane_prefix='%s_%i_%i_%i_%s.dat'
basedir= '/gpfs/alpine/cfd162/scratch/lcheung/AWAKEN/UnstableABL1_precursor2/post_processing/extractpoints/'
HHdir  = basedir+'/timetrace_z90/KP/'
TIdir  = basedir+'/timetrace_z60/KP/'
turbinenames    = [line.strip() for line in open(basedir+'/KPturbnames.txt', 'r')]
turbinelist_ixy = np.loadtxt(basedir+'/KP_ixyz.dat')
Nturbs = len(turbinelist_ixy) #41
zhh    = 90.0
zti    = 60.0

In [4]:
# Load all of the data
alldat91 = []
alldat60 = []
for i, txy in enumerate(turbinelist_ixy[:Nturbs]):
    HHfilename = HHdir+'/'+HHplane_prefix%(turbinenames[i], int(txy[0]), int(txy[1]), int(txy[2]), 'z90')
    TIfilename = TIdir+'/'+HHplane_prefix%(turbinenames[i], int(txy[0]), int(txy[1]), int(txy[2]), 'z60')
    alldat91.append(np.loadtxt(HHfilename))
    alldat60.append(np.loadtxt(TIfilename))
    #alldat91.append(np.loadtxt(HHdir+HHplane_prefix%(txy[0],txy[1],zhh)))
    #alldat60.append(np.loadtxt(TIdir+HHplane_prefix%(txy[0],txy[1],zti)))

In [5]:
def getMeanSTD(alldat, tavg):
    meanvec=[]
    stdvec=[]
    maxvec=[]
    minvec=[]
    for dat in alldat:
        if len(tavg)==0:
            tavg=[np.min(dat[:,0])-9999, np.max(dat[:,0])+999]
        filtert=((tavg[0]<=dat[:,0]) & (dat[:,0] <= tavg[1]))
        filterdat=dat[filtert,:]
        meanvec.append(np.mean(filterdat[:,1:],axis=0))
        maxvec.append(np.max(filterdat[:,1:],axis=0))
        minvec.append(np.min(filterdat[:,1:],axis=0))
        stdvec.append(np.std(filterdat[:,1:],axis=0))
    #return np.mean(meanvec,axis=0), np.mean(stdvec,axis=0), np.max(maxvec,axis=0), np.min(minvec,axis=0)
    return np.array(meanvec), np.array(stdvec), np.array(maxvec), np.array(minvec)

def getUAlphaTI(dat60, dat91, tavg):
    Uvec60, stdvec60, max60, min60 = getMeanSTD(dat60,tavg)
    Uvec91, stdvec91, max91, min91 = getMeanSTD(dat91,tavg)
    #print(Uvec60[:,:2])
    #print(len(np.linalg.norm(Uvec60[:,:2], axis=0)))
    Uh60  = np.mean(np.linalg.norm(Uvec60[:,7:9], axis=1))
    Uh91  = np.mean(np.linalg.norm(Uvec91[:,7:9], axis=1))
    #stdvec60 = np.array(np.array(stdvec60))
    sh60  = np.sqrt(stdvec60[:,7]**2 + stdvec60[:,8]**2)
    TI60  = sh60/np.linalg.norm(Uvec60[:,7:9], axis=1) #/Uh60
    TI60  = [np.sqrt(std60[i][7]**2 + std60[i][8]**2)/np.linalg.norm(Uvec60[i,7:9]) for i in range(len(Uvec60))]
    alpha = np.log(Uh91/Uh60)/np.log(float(zhh)/float(zti))
    return np.mean(Uh91), np.mean(alpha), np.max(TI60) #np.mean(TI60)

In [6]:
twin  = 900 #1200 #900 
tstart= np.arange(15000,18411-twin,25)

Uhvec=[]
alphavec=[]
TIvec=[]

for t1 in tstart:
    tavg=[t1, t1+twin]
    #U91, std91, max91, min91 = getMeanSTD(alldat91,tavg)
    U60, std60, max60, min60 = getMeanSTD(alldat60,tavg)
    Uh60  = np.mean(np.linalg.norm(U60[:,7:9], axis=1))
    #print(std60.shape)
    #Uh91     = np.linalg.norm(U91[:2])
    #Uh60     = np.linalg.norm(U60[:2])
    #alpha    = np.log(Uh91/Uh60)/np.log(zhh/zti)
    #sigmah60 = np.sqrt(std60[0]**2 + std60[1]**2)
    Uh91, alpha, TI = getUAlphaTI(alldat60, alldat91, tavg)
    #print(alpha)
    Uhvec.append(Uh91)
    alphavec.append(alpha)
    TIvec.append(TI)
    #TIvec.append(sigmah60/Uh60)
    #print("%i %f %f %f"%(t1,Uh91, alpha, sigmah60/Uh60))
    
    print("%i %f %f %f %f"%(t1,Uh91, alpha, TI, 
                            np.sqrt(std60[40][7]**2 + std60[40][8]**2)/np.linalg.norm(U60[40,7:9])))

15000 9.114470 0.073092 0.145723 0.109445
15025 9.111088 0.072932 0.140378 0.109338
15050 9.106740 0.072770 0.143718 0.112252
15075 9.104901 0.073060 0.146703 0.111875
15100 9.101679 0.073617 0.147244 0.110706
15125 9.098349 0.073837 0.149434 0.110108
15150 9.092098 0.073789 0.149258 0.110526
15175 9.087028 0.073878 0.150202 0.111279
15200 9.079658 0.073711 0.147653 0.109842
15225 9.070372 0.073194 0.145832 0.110153
15250 9.062870 0.073041 0.145537 0.109444
15275 9.057183 0.072809 0.146619 0.106236
15300 9.051231 0.071742 0.146937 0.108383
15325 9.045698 0.071569 0.147296 0.106370
15350 9.040211 0.071965 0.147672 0.103309
15375 9.036867 0.071519 0.147927 0.101364
15400 9.034723 0.071449 0.147458 0.097110
15425 9.032622 0.071279 0.144047 0.093814
15450 9.030121 0.071070 0.141504 0.094179
15475 9.023715 0.070987 0.144999 0.096023
15500 9.019662 0.071247 0.147594 0.099342
15525 9.017759 0.071002 0.149346 0.100550
15550 9.018358 0.071241 0.152705 0.103761
15575 9.019066 0.070918 0.152805 0